In [4]:
import pandas as pd
from datasets import Dataset

In [6]:
df = pd.read_json('../data/NQ-nli-gpt35.json')
train_df = pd.read_json('learned-NQ-train.jsonl', orient='records', lines=True)
test_df = pd.read_json('learned-NQ-test.jsonl', orient='records', lines=True)

In [7]:
df.set_index(['qid', 'system'], inplace=True)

In [8]:
nli_df = df

In [9]:
# next, convert train_df to llama finetune format
template = """<s>[INST] Here is a question, a set of golden answers (split with /), an AI-generated answer.
Can you judge whether the AI-generated answer is correct according to the question and golden answers, simply answer Yes or No.

Question: {question}

Golden answers: {golden_answer}

AI answer: {system}

Can golden answers be inferred from AI answer: {a2astar}

Can AI answer be inferred from golden answers: {astar2a}
[/INST] {assistant_text} </s>"""

def gen_item(df):
    for i, row in df.iterrows():
        item = row.to_dict()
        try:
            nli = nli_df.loc[int(row['qid']), row['system']]
            a2astar = nli['a2astar'].lower()
            astar2a = nli['astar2a'].lower()
        except KeyError:
            print("unk")
            a2astar = 'unknown'
            astar2a = 'unknown'
        text = template.format(
            question=item['question'],
            golden_answer=item['golden_answer'],
            system=item['system_answer'],
            a2astar=a2astar,
            astar2a=astar2a,
            assistant_text='Yes' if item['golden_judge'] == 1 else 'No',
        )
        yield {'text': text}


In [10]:
dataset = Dataset.from_generator(gen_item, gen_kwargs={'df': train_df})

Generating train split: 0 examples [00:00, ? examples/s]

unk
unk
unk
unk
unk
unk
unk
unk
unk
unk
unk
unk
unk
unk
unk
unk
unk
unk
unk
unk
unk
unk
unk
unk
unk
unk


In [26]:
dataset.save_to_disk('llama-nli-NQ-train')

Saving the dataset (0/1 shards):   0%|          | 0/7550 [00:00<?, ? examples/s]